![alt text](metrics_phenotyping.png)

In [1]:
import torch
import torch.utils.data
from torch.utils.data import TensorDataset

import argparse
import time
import gc
import random

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from tqdm.notebook import tqdm

import wandb

from sklearn.metrics import  balanced_accuracy_score

In [2]:
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'NVIDIA GeForce RTX 4060 Ti')

In [3]:
from net import GNNStack
from utils_todynet_multiclass import AverageMeter, accuracy, log_msg, get_default_train_val_test_loader

In [4]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [5]:
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=None, gamma=2):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma

#     def forward(self, inputs, targets):
#         ce_loss = F.cross_entropy(inputs, targets, reduction='none')
#         pt = torch.exp(-ce_loss)
#         loss = (self.alpha[targets] * (1 - pt) ** self.gamma * ce_loss).mean()
#         return loss

## aruguments

In [6]:
args = {
    'arch': 'dyGIN2d', #what other models I can put here?? dyGCN2d, dyGIN2d
    'dataset': 'Phenotyping', # "AtrialFibrillation" # 'Mortality', # 'MIMIC3'
    'num_layers': 2,  # the number of GNN layers  3
    'groups': 16,  # the number of time series groups (num_graphs)
    'pool_ratio': 0.1,  # the ratio of pooling for nodes
    'kern_size': [3,3],  # list of time conv kernel size for each layer [9,5,3]
    'in_dim': 32,  # input dimensions of GNN stacks
    'hidden_dim': 32,  # hidden dimensions of GNN stacks
    'out_dim': 32,  # output dimensions of GNN stacks
    'workers': 0,  # number of data loading workers
    'epochs': 30,  # number of total epochs to run
    'batch_size': 4,  # mini-batch size, this is the total batch size of all GPUs
    'val_batch_size': 4,  # validation batch size
    'lr': 0.000095,  # initial learning rate
    'weight_decay': 1e-4,  # weight decay
    'evaluate': False,  # evaluate model on validation set
    'seed': 42,  # seed for initializing training
    'gpu': 0,  # GPU id to use
    'use_benchmark': True,  # use benchmark
    'tag': 'date',  # the tag for identifying the log and model files
    'weight':'alldata'
}

In [7]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="phenotyping",
    
    # track hyperparameters and run metadata
    config=args
)

wandb: Currently logged in as: shashankyadav321. Use `wandb login --relogin` to force relogin


In [8]:
# train_dataset = TensorDataset(data_train, label_train)
# val_dataset   = TensorDataset(data_val, label_val)

In [9]:
# train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=args['batch_size'],shuffle=True, num_workers=args['workers'], pin_memory=True)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args['val_batch_size'], shuffle=False,num_workers=args['workers'],pin_memory=True)

In [10]:
# def main():
#     # args = parser.parse_args()
    
#     # args.kern_size = [ int(l) for l in args.kern_size.split(",") ]

#     # if args.seed is not None:
#     random.seed(args['seed'])
#     torch.manual_seed(args['seed'])

#     main_work(args)

In [11]:
def main_work(args):
    
    random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    
    
    # init acc
    best_acc1 = 0
    best_ba  = 0
    # best_pr   = 0
    
    if args['tag'] == 'date':
        local_date = time.strftime('%m.%d %H:%M', time.localtime(time.time()))
        args['tag'] = local_date

    log_file = 'log/{}_gpu{}_{}_{}_exp.txt'.format(args['tag'], args['gpu'], args['arch'], args['dataset'])
    
    
    if args['gpu'] is not None:
        print("Use GPU: {} for training".format(args['gpu']))


    # dataset
    train_loader, val_loader, num_nodes, seq_length, num_classes = get_default_train_val_test_loader(args)
    
    print('features / nodes', num_nodes,'total time graphs',seq_length,'classes', num_classes)
    
    # training model from net.py
    model = GNNStack(gnn_model_type=args['arch'], num_layers=args['num_layers'], 
                     groups=args['groups'], pool_ratio=args['pool_ratio'], kern_size=args['kern_size'], 
                     in_dim=args['in_dim'], hidden_dim=args['hidden_dim'], out_dim=args['out_dim'], 
                     seq_len=seq_length, num_nodes=num_nodes, num_classes=num_classes)

    # print & log
    log_msg('epochs {}, lr {}, weight_decay {}'.format(args['epochs'], args['lr'], args['weight_decay']), log_file)
    
    log_msg(str(args), log_file)


    # determine whether GPU or not
    if not torch.cuda.is_available():
        print("Warning! Using CPU!!!")
    elif args['gpu'] is not None:
        torch.cuda.set_device(args['gpu'])

        # collect cache
        gc.collect()
        torch.cuda.empty_cache()

        model = model.cuda(args['gpu'])
        if args['use_benchmark']:
            cudnn.benchmark = True
        print('Using cudnn.benchmark.')
    else:
        print("Error! We only have one gpu!!!")


    # define loss function(criterion) and optimizer
    weights = np.array([1.48,2.15,3.77,1.0,4.51,3.54,10.20,61.57,1.90,14.27,4.93,3.17,23.72,130.0,19.39])  ### according to whole data
    # weights = np.array([1.44,2.15,3.71,1.00,4.54,3.56,9.90,58.43,1.93,14.10,4.97,3.15,20.97,106.70,18.88]) ### according to train
    class_weights = torch.tensor(weights, dtype=torch.float).cuda(args['gpu'])   
    criterion = nn.CrossEntropyLoss(weight=class_weights).cuda(args['gpu'])
    # criterion = nn.CrossEntropyLoss().cuda(args['gpu'])

    
    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
    
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)

    # validation
    if args['evaluate']:
        validate(val_loader, model, criterion, args)
        return

    # train & valid
    print('****************************************************')
    print('Dataset: ', args['dataset'])

    dataset_time = AverageMeter('Time', ':6.3f')

    loss_train = []
    acc_train = []
    loss_val = []
    acc_val = []
    epoches = []
    
    ###### 4 more lists to have values
    ba_train = []
    # pr_train  = []
    
    ba_val   = []
    # pr_val    = []

    end = time.time()
    for epoch in tqdm(range(args['epochs'])):
        epoches += [epoch]

        # train for one epoch
        acc_train_per, loss_train_per, output_train_per, target_train_per = train(train_loader, model, criterion, optimizer, lr_scheduler, args)
        
        acc_train += [acc_train_per]
        loss_train += [loss_train_per]
        # calculate metric
        # print(len(target_train_per),len(output_train_per))
        
        # ohe_train = torch.nn.functional.one_hot(target_train_per, num_classes=15)
        
        ba_value_train = balanced_accuracy_score(target_train_per, output_train_per)
        # auc_pr_value_train = average_precision_score(target_train_per, output_train_per)
        #new code
        ba_train += [ba_value_train]
        # pr_train  += [auc_pr_value_train]

        msg = f'TRAIN, epoch {epoch}, train_loss {loss_train_per}, train_bacc {ba_value_train}'

        print(f'TRAIN, epoch {epoch}, train_loss {loss_train_per:.5f}, train_bacc {ba_value_train:.5f}')
        log_msg(msg, log_file)

        
        # evaluate on validation set
        acc_val_per, loss_val_per, output_val_per, target_val_per = validate(val_loader, model, criterion, args)

        acc_val  += [acc_val_per]
        loss_val += [loss_val_per]
        #calculate metric
        # calculate metric
        # print(len(target_val_per),len(output_val_per))
        ba_value_val = balanced_accuracy_score(target_val_per, output_val_per)
        # auc_pr_value_val = average_precision_score(target_val_per, output_val_per)
        #new code

        msg = f'VAL, epoch {epoch}, val_loss {loss_val_per}, val_bacc {ba_value_val}'
        
        print(f'VAL, epoch {epoch}, val_loss {loss_val_per:.5f}, val_bacc {ba_value_val:.5f}')
        
        log_msg(msg, log_file)

        
        
        # remember best acc
        best_acc1 = max(acc_val_per, best_acc1)
        
        best_ba = max(ba_value_val, best_ba)
        
        # best_pr  = max(auc_pr_value_val, best_pr)
        
        wandb.log({"train_loss": loss_train_per, "train_ba": ba_value_train, "val_loss": loss_val_per, "val_ba": ba_value_val, "best_val_ba": best_ba})
        
    wandb.finish()
    # measure elapsed time
    dataset_time.update(time.time() - end)

    # log & print the best_acc
    msg = f'\n\n * BEST_ACC: {best_acc1}\n * TIME: {dataset_time}\n'
    log_msg(msg, log_file)

    print(f' * best_acc1: {best_acc1}, best_ba: {best_ba}')
    print(f' * time: {dataset_time}')
    print('****************************************************')


    # collect cache
    gc.collect()
    torch.cuda.empty_cache()


def train(train_loader, model, criterion, optimizer, lr_scheduler, args):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc', ':6.2f')
    # met_roc = AverageMeter('ROC', ':6.2f')
    # met_pr = AverageMeter('PR', ':6.2f')
    
    output_list = []
    target_list = [] 

    # switch to train mode
    model.train()

    for count, (data, label) in enumerate(train_loader):

        # data in cuda
        data = data.cuda(args['gpu']).type(torch.float)
        label = label.cuda(args['gpu']).type(torch.long)

        # compute output
        output = model(data)
        
        # print(output, label)
    
        loss = criterion(output, label)

        # measure accuracy and record loss
        acc1 = accuracy(output, label, topk=(1, 1))
        
        output_np = np.argmax(torch.softmax(output, dim=1).detach().cpu().numpy(), axis=1).tolist()
        
        target_np = label.detach().cpu().numpy().tolist()
        
        # print(output_np, target_np)
        # break
        
        losses.update(loss.item(), data.size(0))
        top1.update(acc1[0], data.size(0))
        
        # met_roc.update(roc, data.size(0))
        # met_pr.update(pr, data.size(0))
        output_list += output_np
        target_list += target_np

        # compute gradient and do Adam step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    lr_scheduler.step(top1.avg)

    return top1.avg, losses.avg, output_list, target_list


def validate(val_loader, model, criterion, args):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    # met_roc = AverageMeter('ROC', ':6.2f')
    # met_pr = AverageMeter('PR', ':6.2f')
    output_list = []
    target_list = [] 
    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for count, (data, label) in enumerate(val_loader):
            if args['gpu'] is not None:
                data = data.cuda(args['gpu'], non_blocking=True).type(torch.float)
            if torch.cuda.is_available():
                label = label.cuda(args['gpu'], non_blocking=True).type(torch.long)

            # compute output
            output = model(data)

            loss = criterion(output, label)
            
            output_np = np.argmax(torch.softmax(output, dim=1).detach().cpu().numpy(), axis=1).tolist()
            target_np = label.detach().cpu().numpy().tolist()

            # measure accuracy and record loss
            acc1 = accuracy(output, label, topk=(1, 1))
            losses.update(loss.item(), data.size(0))
            top1.update(acc1[0], data.size(0))
            
            output_list += output_np
            target_list += target_np
            
            # met_roc.update(roc, data.size(0))
            # met_pr.update(pr, data.size(0))

    return top1.avg, losses.avg, output_list, target_list

In [ ]:
main_work(args)

Use GPU: 0 for training
features / nodes 231 total time graphs 288 classes 15
Using cudnn.benchmark.
****************************************************
Dataset:  Phenotyping


  0%|          | 0/30 [00:00<?, ?it/s]

TRAIN, epoch 0, train_loss 2.50814, train_bacc 0.11819
VAL, epoch 0, val_loss 2.23589, val_bacc 0.16281
TRAIN, epoch 1, train_loss 2.30127, train_bacc 0.15800
VAL, epoch 1, val_loss 2.16416, val_bacc 0.17497
TRAIN, epoch 2, train_loss 2.21400, train_bacc 0.17700
VAL, epoch 2, val_loss 2.01315, val_bacc 0.21130
TRAIN, epoch 3, train_loss 2.15260, train_bacc 0.19544
VAL, epoch 3, val_loss 1.97251, val_bacc 0.22864
TRAIN, epoch 4, train_loss 2.10911, train_bacc 0.20672
VAL, epoch 4, val_loss 1.90936, val_bacc 0.25665
TRAIN, epoch 5, train_loss 2.04112, train_bacc 0.23312
VAL, epoch 5, val_loss 1.93507, val_bacc 0.24104
TRAIN, epoch 6, train_loss 1.99579, train_bacc 0.24713
VAL, epoch 6, val_loss 1.78394, val_bacc 0.31169
TRAIN, epoch 7, train_loss 1.95698, train_bacc 0.26932
VAL, epoch 7, val_loss 1.76622, val_bacc 0.31326
TRAIN, epoch 8, train_loss 1.91306, train_bacc 0.28115
VAL, epoch 8, val_loss 1.74959, val_bacc 0.32288
TRAIN, epoch 9, train_loss 1.87128, train_bacc 0.29140
VAL, epoc

In [ ]:
#TODO ignite metrics